# Versão 07.09.2023

In [ ]:
import os
import cv2
import pickle
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
import face_recognition
from sklearn import svm
import time
import matplotlib.pyplot as plt

class ReconhecimentoFacialApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reconhecimento Facial Yale")

        # Centralize a janela principal
        window_width = 800
        window_height = 600
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        self.root.geometry(f"{window_width}x{window_height}+{x}+{y}")

        self.root.configure(bg="black")
        
        self.criar_widgets()

    def criar_widgets(self):
        self.criar_frame_titulo()
        self.criar_frame_botoes()
        self.criar_frame_resultado()

    def criar_frame_titulo(self):
        self.frame_titulo = tk.Frame(self.root, bg="black")
        self.frame_titulo.pack(pady=20)
        
        self.label_titulo = tk.Label(self.frame_titulo, text="Sistema de Reconhecimento Facial Yale", font=("Helvetica", 20), bg="black", fg="white")
        self.label_titulo.pack()

    def criar_frame_botoes(self):
        self.frame_botoes = tk.Frame(self.root, bg="black")
        self.frame_botoes.pack()

        self.botao_treinamento = tk.Button(self.frame_botoes, text="Treinamento", font=("Helvetica", 18), command=self.realizar_treinamento, bg="aquamarine", fg="black")
        self.botao_treinamento.pack(side=tk.LEFT, padx=10)

        self.botao_validacao_teste = tk.Button(self.frame_botoes, text="Validação/Teste", font=("Helvetica", 18), command=self.realizar_validacao_teste, bg="aquamarine", fg="black")
        self.botao_validacao_teste.pack(side=tk.LEFT, padx=10)
        
    def criar_frame_resultado(self):
        self.frame_resultado = tk.Frame(self.root, bg="black", highlightbackground="aquamarine", highlightthickness=2)
        self.frame_resultado.pack(pady=20)
        
        self.label_imagem_validacao = tk.Label(self.frame_resultado, bg="black", padx=10, pady=10)
        self.label_imagem_validacao.pack()

        self.label_mensagem_reconhecimento = tk.Label(self.frame_resultado, text="", bg="black", fg="white", font=("Helvetica", 16))
        self.label_mensagem_reconhecimento.pack()

        self.frame_icones = tk.Frame(self.frame_resultado, bg="black")
        self.frame_icones.pack(pady=10)

        self.fig, self.ax = plt.subplots()  
        self.ax.set_xlabel("Treinamento")
        self.ax.set_ylabel("Acurácia")
        self.acuracia_data = []

    def realizar_treinamento(self):
        yale_encodings = []
        yale_labels = []
        imagens_com_erro = []
        total_imagens = 0
        imagens_treinadas = 0
        dataset_path = 'src/yalefaces'

        for subject_folder in os.listdir(dataset_path):
            subject_folder_path = os.path.join(dataset_path, subject_folder)
            if os.path.isdir(subject_folder_path):
                image_count = 0
                for img_name in os.listdir(subject_folder_path):
                    if img_name.endswith(".gif"):
                        image_count += 1
                        if image_count <= 7:
                            total_imagens += 1
                            identity = subject_folder
                            image = face_recognition.load_image_file(os.path.join(subject_folder_path, img_name))
                            
                            # Verifique se a imagem está em escala de cinza
                            if len(image.shape) == 3 and image.shape[2] == 3:
                                # Converta a imagem para tons de cinza
                                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                            
                            # Equalização de histograma
                            image = cv2.equalizeHist(image)

                            # Converta a imagem de tons de cinza de volta para RGB
                            image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

                            # Detecte os rostos na imagem
                            face_locations = face_recognition.face_locations(image_rgb)
                            if len(face_locations) == 1:
                                face_location = face_locations[0]
                                top, right, bottom, left = face_location
                                face_image = image_rgb[top:bottom, left:right]
                                
                                # Codifique o rosto
                                face_encoding = face_recognition.face_encodings(face_image)
                                if len(face_encoding) > 0:
                                    yale_encodings.append(face_encoding[0])
                                    yale_labels.append(identity)
                                    imagens_treinadas += 1
                                    print(f"Face cadastrada: {identity}")
                                else:
                                    imagens_com_erro.append(os.path.join(subject_folder_path, img_name))
                                    print(f"Erro na imagem: {os.path.join(subject_folder_path, img_name)}")
                            else:
                                imagens_com_erro.append(os.path.join(subject_folder_path, img_name))
                                print(f"Erro na imagem: {os.path.join(subject_folder_path, img_name)}")
        
        print(f"Total de imagens processadas: {total_imagens}")
        print(f"Total de imagens treinadas: {imagens_treinadas}")
        print(f"Imagens com erro: {len(imagens_com_erro)}")
        
        # Salve as codificações e rótulos em arquivos para uso posterior
        np.save('yale_encodings.npy', yale_encodings)
        np.save('yale_labels.npy', yale_labels)

    def realizar_validacao_teste(self):
        image_path = filedialog.askopenfilename(title="Selecionar Imagem", filetypes=[("Imagens", "*.gif")])
        if image_path:
            # Remova o frame de ícones existente e crie um novo
            self.frame_icones.destroy()
            self.frame_icones = tk.Frame(self.root, bg="black")
            self.frame_icones.pack()
            self.validar_reconhecimento(image_path)
    
    def validar_reconhecimento(self, image_path):
        if not os.path.exists("modelo_classificador.pkl"):
            messagebox.showerror("Erro", "Modelo não treinado. Realize o treinamento antes de fazer a validação/teste.")
            return

        with open("modelo_classificador.pkl", "rb") as f:
            svm_classifier = pickle.load(f)

        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        if not face_encodings:
            #messagebox.showerror("Erro", "Nenhuma face foi encontrada na imagem.")
            self.label_mensagem_reconhecimento.config(text="Pessoa não reconhecida")
            self.mostrar_icone_x()
            # Mostrar a imagem original na label
            img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tk = ImageTk.PhotoImage(img_pil)
            self.label_imagem_validacao.config(image=img_tk)
            self.label_imagem_validacao.image = img_tk
            return

        predicted_labels = []
        for face_encoding in face_encodings:
            _label = svm_classifier.predict([face_encoding])
            predicted_labels.append(_label[0])

        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)
        self.label_imagem_validacao.config(image=img_tk)
        self.label_imagem_validacao.image = img_tk

        predicted_names = ", ".join(predicted_labels)
        if predicted_names:
            self.label_mensagem_reconhecimento.config(text=f"Pessoa Reconhecida: {predicted_names}")
            self.mostrar_icone_verificado()
        else:
            self.label_mensagem_reconhecimento.config(text="Pessoa não reconhecida")
            self.mostrar_icone_x()

        self.mostrar_resultado_reconhecimento(predicted_names)


    def mostrar_resultado_reconhecimento(self, predicted_names):
        self.frame_icones.destroy()  # Remover o frame existente

        self.frame_icones = tk.Frame(self.frame_resultado, bg="black")
        self.frame_icones.pack(pady=10)

        if predicted_names:
            self.mostrar_icone_verificado()
        else:
            self.mostrar_icone_x()
            

    def mostrar_icone_verificado(self):
        # Crie e mostre o ícone de verificação
        icone_verificado = Image.open("icone_verificado.png")
        icone_verificado = icone_verificado.resize((100, 100))
        icone_verificado = ImageTk.PhotoImage(icone_verificado)
        label_icone = tk.Label(self.frame_icones, image=icone_verificado, bg="black")
        label_icone.image = icone_verificado
        label_icone.pack()

    def mostrar_icone_x(self):
        # Crie e mostre o ícone "X"
        icone_x = Image.open("icone_x.png")
        icone_x = icone_x.resize((100, 100))
        icone_x = ImageTk.PhotoImage(icone_x)
        label_icone = tk.Label(self.frame_icones, image=icone_x, bg="black")
        label_icone.image = icone_x
        label_icone.pack()


    def plot_acuracia(self):
        self.ax.clear()
        self.ax.set_xlabel("Treinamento")
        self.ax.set_ylabel("Acurácia")

        # Ajuste o tamanho da figura
        self.fig.set_size_inches(10, 6)  # Escolha as dimensões desejadas

        # Personalize o intervalo do eixo x (por exemplo, de 0 a 100)
        self.ax.set_xlim(0, len(self.acuracia_data) + 1)
        self.ax.set_xticks(range(0, len(self.acuracia_data) + 1, 10))  # Personalize os intervalos dos ticks se necessário

        self.ax.plot(range(1, len(self.acuracia_data) + 1), self.acuracia_data, marker='o')
        self.ax.set_ylim([0, 1])
        self.ax.grid(True)
        self.fig.canvas.draw()
    
        plt.show()


if __name__ == "__main__":
    root = tk.Tk()
    app = ReconhecimentoFacialApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\env_saad\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Leonardo\AppData\Local\Temp\ipykernel_3772\2670590639.py", line 109, in realizar_treinamento
    face_encoding = face_recognition.face_encodings(face_image)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\envs\env_saad\Lib\site-packages\face_recognition\api.py", line 214, in face_encodings
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for raw_landmark_set in raw_landmarks]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\envs\env_saad\Lib\site-packages\face_recognition\api.py", line 214, in <listcomp>
    return [np.array(face_enc